In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 5       # number of variables
num_ineq = 5      # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msAckley
model = msAckley(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_exact_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:21<00:00,  7.07it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   8.461999e+00               0.0      0.140247
std    3.616767e+00               0.0      0.077084
min    4.440892e-16               0.0      0.059051
25%    5.855553e+00               0.0      0.107038
50%    8.476709e+00               0.0      0.121865
75%    1.114599e+01               0.0      0.146774
max    1.617466e+01               0.0      1.238860
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_heur_rnd_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:45<00:00,  1.07s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03       1000.000000   1000.000000
mean   8.167062e+00          0.015945      1.060173
std    3.658902e+00          0.019701      2.116884
min    4.440892e-16          0.000000      0.069687
25%    5.688431e+00          0.000000      0.899114
50%    8.217893e+00          0.009834      1.008161
75%    1.073432e+01          0.022586      1.115101
max    1.617466e+01          0.124860     59.588239
Number of infeasible solution: 711


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_heur_n1_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:32<00:00, 10.82it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.354871e+01               0.0      0.091665
std    4.998688e+00               0.0      0.033452
min    4.440892e-16               0.0      0.059494
25%    1.027576e+01               0.0      0.076484
50%    1.380086e+01               0.0      0.077584
75%    1.820365e+01               0.0      0.090267
max    1.999996e+01               0.0      0.248882
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-5             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmAckley
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmAckley(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 135.19
Epoch 1, Validation Loss: nan
Epoch 2, Validation Loss: nan
Epoch 3, Validation Loss: nan
Epoch 4, Validation Loss: nan
Epoch 5, Validation Loss: nan
Epoch 6, Validation Loss: nan
Epoch 7, Validation Loss: nan
Epoch 8, Validation Loss: nan
Epoch 9, Validation Loss: nan
Epoch 10, Validation Loss: nan
Epoch 11, Validation Loss: nan
Epoch 12, Validation Loss: nan
Epoch 13, Validation Loss: nan
Epoch 14, Validation Loss: nan
Epoch 15, Validation Loss: nan
Epoch 16, Validation Loss: nan
Epoch 17, Validation Loss: nan
Epoch 18, Validation Loss: nan
Epoch 19, Validation Loss: nan
Epoch 20, Validation Loss: nan
Epoch 21, Validation Loss: nan
Epoch 22, Validation Loss: nan
Epoch 23, Validation Loss: nan
Epoch 24, Validation Loss: nan
Epoch 25, Validation Loss: nan
Epoch 26, Validation Loss: nan
Epoch 27, Validation Loss: nan
Epoch 28, Validation Loss: nan
Epoch 29, Validation Loss: nan
Epoch 30, Validation Loss: nan
Epoch 31, Validation Loss: nan
Epoch 32, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_lr_5-5.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 285.95it/s]


       Obj Val  Constraints Viol  Elapsed Time
count      0.0            1000.0   1000.000000
mean       NaN               0.0      0.002487
std        NaN               0.0      0.000771
min        NaN               0.0      0.000999
25%        NaN               0.0      0.002001
50%        NaN               0.0      0.002102
75%        NaN               0.0      0.003000
max        NaN               0.0      0.014007
Number of infeasible solution: 0


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmAckley
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmAckley(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 126.15
Epoch 1, Validation Loss: nan
Epoch 2, Validation Loss: nan
Epoch 3, Validation Loss: nan
Epoch 4, Validation Loss: nan
Epoch 5, Validation Loss: nan
Epoch 6, Validation Loss: nan
Epoch 7, Validation Loss: nan
Epoch 8, Validation Loss: nan
Epoch 9, Validation Loss: nan
Epoch 10, Validation Loss: nan
Epoch 11, Validation Loss: nan
Epoch 12, Validation Loss: nan
Epoch 13, Validation Loss: nan
Epoch 14, Validation Loss: nan
Epoch 15, Validation Loss: nan
Epoch 16, Validation Loss: nan
Epoch 17, Validation Loss: nan
Epoch 18, Validation Loss: nan
Epoch 19, Validation Loss: nan
Epoch 20, Validation Loss: nan
Epoch 21, Validation Loss: nan
Epoch 22, Validation Loss: nan
Epoch 23, Validation Loss: nan
Epoch 24, Validation Loss: nan
Epoch 25, Validation Loss: nan
Epoch 26, Validation Loss: nan
Epoch 27, Validation Loss: nan
Epoch 28, Validation Loss: nan
Epoch 29, Validation Loss: nan
Epoch 30, Validation Loss: nan
Epoch 31, Validation Loss: nan
Epoch 32, Valid

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_lt_5-5.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 274.33it/s]


       Obj Val  Constraints Viol  Elapsed Time
count      0.0            1000.0   1000.000000
mean       NaN               0.0      0.002593
std        NaN               0.0      0.000848
min        NaN               0.0      0.000997
25%        NaN               0.0      0.002000
50%        NaN               0.0      0.002420
75%        NaN               0.0      0.003000
max        NaN               0.0      0.006720
Number of infeasible solution: 0
